<a href="https://colab.research.google.com/github/kiryu-3/Prmn2023_DS/blob/main/Python/Python_Stats/Stats_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 分散分析1

In [36]:
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api as sm
from scipy.stats import f
from decimal import Decimal, ROUND_HALF_UP, ROUND_HALF_EVEN

## 分散分析とは

平均値の差の検定において、3群以上からなるデータに対しては  
（推定の多重性のため）$t$検定を行うことができません。

このような場合にも対応できる、  
「分散の比」を考えて平均値の差を検定する分析手法を**分散分析**といいます。

### 基本用語

* **要因**：データの値に変化を与える要素のこと  
* **因子**：要因の中でも特に、母平均に差をもたらすと考えられることから注目する要因  
* **水準**：1つの要因に含まれる項目のこと
* **〇元配置**：データに含まれる因子の数を表すもの

### 分散分析の種類

分散分析にはいくつかの種類があります。

* 「一元配置分散分析」：1つの因子からデータを分析する方法。  
　　　　　　　　　　　因子に含まれる水準間の平均値の差を見ることができる。  

* 「二元配置分散分析」：2つの因子からなるデータを分析する方法。  
　　　　　　　　　　　各因子における水準間の平均値の差を見ることができる。  
　　　　　　　　　　　また、2つの要因が組み合わさることで現れる相乗効果の有無も確認できる。

* 「多元配置分散分析」：3つ以上の因子からなるデータを分析する方法。

## 一元配置分散分析

分散分析では、「全体平均から因子の各水準の平均値がどの程度ズレているか」に注目します。



ズレは以下の式で表すことができます。

「全体平均から各データのズレ」=「全体平均から各群の平均値のズレ」 + 「各群の平均値から各データのズレ」

これらのズレを二乗和として求めます。

### 分散分析表

分散分析では、**分散分析表**という表を用いて、結果を分かりやすく表現することが多いです。  

一元配置分散分析の分散分析表は以下の通りです。

| 因子 | 平方和 | 自由度 | 平均平方 | F値 |
|:--------:|:--------:|:--------:|:--------:|:--------:|
|要因| | ||
|残差| | ||
|全体| | ||

「平方和」の部分にはそれぞれのズレの二乗和が入ります。

* 「要因」：全体平均から各群の平均値のズレ
* 「残差」：各群の平均値から各データのズレ
* 「全体」：全体平均から各データのズレ

「自由度」は以下のものが入ります。

* 「要因」：因子の水準の個数から1を引いたもの  
* 「残差」：全体の自由度から要因の自由度を引いたもの  
* 「全体」：すべてのデータの個数から1を引いたもの

「平均平方」には「平方和」を「自由度」で割ったものが入ります。


最後に「要因の平均平方」を分子に、「残差の平均平方」を分母にして  
統計量$F$を算出します。

### 演習

まずは[こちら](https://bit.ly/3Y7Hzup)の例題について、分散分析を行ってみます。

In [3]:
df = pd.read_excel("movie_theatre.xlsx")
df.iloc[[0,7,14]]

,地方,映画館の数
0,北海道・東北,21
7,関東,92
14,中部,2


ここで用語について整理します。

* **要因**：「映画館の数」に変化を与える要素のこと。  
　　　　今回の場合は、「地方」のほかに「人口」など様々なものが考えられる。
* **因子**：要因の中でも特に、「映画館の数」の母平均に差をもたらすと考えられる「地方」のこと。  
* **水準**：今回の場合は、7つの地方に分けられるので水準数は「7」。
* **一元配置**：今回は因子が「地方」のみであるので「一元配置」。

### 仮説検定のステップ（再掲）
①帰無仮説と対立仮説を立てる  
②帰無仮説のもとで標本観察を行う  
③帰無仮説を棄却できるかどうかを確認する

#### ① 帰無仮説と対立仮説を立てる  

帰無仮説と対立仮説は以下のようになります。（**片側検定（上側）**になります）  

帰無仮説$H_0$：「各地方の映画館数の母平均は等しい」  
対立仮説$H_1$：「各地方の映画館数の母平均では差がある」

分散分析では「残差のばらつき」に対して「要因のばらつき」が大きいかどうかを検定するため、  
必ず「片側検定」を行います。

#### ② 帰無仮説のもとで標本観察を行う

##### 全体平均と各群の平均値の導出

以下のようになります。

In [4]:
df["映画館の数"].mean()

9.382978723404255

In [5]:
new_df = df.groupby("地方").mean()
new_df = new_df.reindex(index=df["地方"].unique())
new_df = new_df.reset_index()
new_df

,地方,映画館の数
0,北海道・東北,9.285714
1,関東,19.000000
2,中部,8.111111
3,近畿,12.428571
4,中国,5.400000
5,四国,2.250000
6,九州・沖縄,5.875000


##### 全体平均から各データのズレ

このズレの二乗和を求めます。

In [6]:
df["全体平均 - 各データ"] = (df["映画館の数"] - df["映画館の数"].mean())**2
df.iloc[[0,7,14]]

,地方,映画館の数,全体平均 - 各データ
0,北海道・東北,21,134.955183
7,関東,92,6825.572205
14,中部,2,54.508375


In [7]:
round(sum(df["全体平均 - 各データ"]),2)

9085.11

##### 全体平均から各群の平均値のズレ

このズレの二乗和を求めます。

In [8]:
goukei = []
for i in range(len(df["映画館の数"])):
  if df.loc[i]["地方"] in new_df["地方"].unique():
    goukei.append(float(new_df[new_df["地方"]  == df.loc[i]["地方"]]["映画館の数"]))

df["各群の平均値"] = pd.Series(goukei)
df.iloc[[0,7,14]]

,地方,映画館の数,全体平均 - 各データ,各群の平均値
0,北海道・東北,21,134.955183,9.285714
7,関東,92,6825.572205,19.000000
14,中部,2,54.508375,8.111111


In [9]:
df["全体平均 - 各地方の平均"] = (df["映画館の数"].mean() - df["各群の平均値"])**2
df.iloc[[0,7,14]]

,地方,映画館の数,全体平均 - 各データ,各群の平均値,全体平均 - 各地方の平均
0,北海道・東北,21,134.955183,9.285714,0.009460
7,関東,92,6825.572205,19.000000,92.487098
14,中部,2,54.508375,8.111111,1.617647


In [10]:
round(sum(df["全体平均 - 各地方の平均"]),2)

1108.25

##### 各群の平均値から各データのズレ

引き算をすることで求めます。

In [11]:
round(sum(df["全体平均 - 各データ"]),2) - round(sum(df["全体平均 - 各地方の平均"]),2)

7976.860000000001

##### 自由度→平均平方

ここまで分かったことを、分散分析表にまとめていきます。

| 因子 | 平方和 | 自由度 | 平均平方 | F値 |
|:--------:|:--------:|:--------:|:--------:|:--------:|
|要因|1108.25 | ||
|残差|7976.86 | ||
|全体|9085.11 | ||

自由度、平均平方もすぐ求められます。

| 因子 | 平方和 | 自由度 | 平均平方 | F値 |
|:--------:|:--------:|:--------:|:--------:|:--------:|
|要因|1108.25 |6|184.71||
|残差|7976.86 |40|199.42||
|全体|9085.11 |46|||

##### 統計量$F$

要因の平均平方を「分子」、残差の平均平方を「分母」として、  
統計量$F$を算出します。

$$
F = \frac{s^2_1}{s^2_2} = \frac{184.71}{199.42} = 0.926
$$

| 因子 | 平方和 | 自由度 | 平均平方 | F値 |
|:--------:|:--------:|:--------:|:--------:|:--------:|
|要因|1108.25 |6|184.71|0.926|
|残差|7976.86 |40|199.42||
|全体|9085.11 |46|||

#### ③ 帰無仮説を棄却できるかどうかを確認する

有意水準を$5$%に設定します。

($p$**値を求めて**) 棄却が必要かどうか決めます。  

In [21]:
fk = stats.f(dfn=6 , dfd=40).ppf(0.95)
fk

2.335852404791663

In [23]:
nyp = float(Decimal(str(stats.f(dfn=6 , dfd=40).sf(fk)*100)).
            quantize(Decimal('0.001'),rounding=ROUND_HALF_UP))
print(f"有意水準：{nyp}%")    # 右側の棄却域の面積

nkp = float(Decimal(str(stats.f(dfn=6 , dfd=40).sf(0.926)*100)).
            quantize(Decimal('0.000001'),rounding=ROUND_HALF_UP))
print(f"p値：{nkp}%")

有意水準：5.0%
p値：48.682816%


In [ ]:
print("p値が有意水準より大きくなったので、帰無仮説を棄却せず、対立仮説を採択しない")
print("したがって、地方によって映画館の数の母平均に差があるとはいえない")

### Pythonによる一元配置分散分析

と、ここまで結構ステップを踏んできましたが、  
Pythonでは簡単に分散分析表を求めることができます。

In [50]:
df = pd.read_excel("movie_theatre.xlsx")
df.iloc[[0,7,14]]

,地方,映画館の数
0,北海道・東北,21
7,関東,92
14,中部,2


In [51]:
model = ols('映画館の数 ~ 地方', data=df).fit()
anova = sm.stats.anova_lm(model, typ=2)
anova  

,sum_sq,df,F,PR(>F)
地方,1108.249637,6.0,0.926221,0.48668
Residual,7976.856746,40.0,NaN,NaN


$F$値と$P$値を簡単に求めることができました。

### 都道府県データで分散分析

男性の平均睡眠時間について、地方間で差があるかどうかを検定します。

`todohuken_kaidata.csv`をインポートしてください。

In [60]:
df = pd.read_csv("todohuken_kaidata.csv")
df = df[["地方","15歳以上の平均睡眠時間（男）"]]
df.head(3)

,地方,15歳以上の平均睡眠時間（男）
0,北海道地方,473
1,東北地方,486
2,東北地方,477


北海道は東北地方と混ぜてしまいます。

In [61]:
for idx,row in df.iterrows():  # rowには各行のSeriesが⼊る
 if row["地方"] in ["北海道地方","東北地方"]: 
  row["地方"] = "北海道・東北地方"
  df.iloc[idx] = row

df.head(3)

,地方,15歳以上の平均睡眠時間（男）
0,北海道・東北地方,473
1,北海道・東北地方,486
2,北海道・東北地方,477


エラーを解消する目的で、カラム名を変更します。

In [62]:
df = df.rename(columns={'地方': 'region', 
                   '15歳以上の平均睡眠時間（男）': 'average_sleep_time_male'})
df.head(3)

,region,average_sleep_time_male
0,北海道・東北地方,473
1,北海道・東北地方,486
2,北海道・東北地方,477


In [63]:
model = ols('average_sleep_time_male~region', data=df).fit()
anova = sm.stats.anova_lm(model, typ=2)
anova

,sum_sq,df,F,PR(>F)
region,1601.791734,6.0,8.445456,0.000006
Residual,1264.421032,40.0,NaN,NaN


$P$値がとても小さくなったので、帰無仮説を棄却できそうです。  

よって、地方ごとに平均睡眠時間に差があるといえます。

### 多重比較法

かなり難しいので、説明は省略します。
（[こちら](https://bellcurve.jp/statistics/blog/14439.html)などに解説があります）

具体的にどの群の間に差異が存在するのかを検定することができます。


In [64]:
print(pairwise_tukeyhsd(df["average_sleep_time_male"], df["region"]))

   Multiple Comparison of Means - Tukey HSD, FWER=0.05    
 group1   group2  meandiff p-adj   lower    upper   reject
----------------------------------------------------------
    中国地方     中部地方  -2.3111    0.9 -12.0426   7.4203  False
    中国地方     九州地方    0.175    0.9  -9.7713  10.1213  False
    中国地方 北海道・東北地方   8.2286 0.1867  -1.9873  18.4445  False
    中国地方     四国地方      2.8    0.9  -8.9038  14.5038  False
    中国地方     近畿地方  -5.7714 0.5752 -15.9873   4.4445  False
    中国地方     関東地方 -11.4857 0.0188 -21.7016  -1.2698   True
    中部地方     九州地方   2.4861    0.9  -5.9916  10.9638  False
    中部地方 北海道・東北地方  10.5397 0.0101   1.7472  19.3321   True
    中部地方     四国地方   5.1111 0.7102  -5.3732  15.5954  False
    中部地方     近畿地方  -3.4603  0.874 -12.2528   5.3321  False
    中部地方     関東地方  -9.1746 0.0359 -17.9671  -0.3822   True
    九州地方 北海道・東北地方   8.0536  0.108  -0.9761  17.0832  False
    九州地方     四国地方    2.625    0.9   -8.059   13.309  False
    九州地方     近畿地方  -5.9464 0.4067 -14.9761   3.0832  Fal

有意差ありの場合、項目`reject`がTrueになるようです。  
Trueになった項目を以下にまとめます。

* 関東地方 ⇔ 北海道・東北地方
* 関東地方 ⇔ 中部地方
* 関東地方 ⇔ 中国地方
* 関東地方 ⇔ 四国地方
* 関東地方 ⇔ 九州地方
* 中部地方 ⇔ 北海道・東北地方
* 近畿地方 ⇔ 北海道・東北地方

なんとなく都心と田舎の特徴が出ている気がしますね。